In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Fehlerkorrektur- und Fehlerunterdrückungstechniken

> **Note:** Die Beta-Version eines neuen Ausführungsmodells ist jetzt verfügbar. Das Directed Execution Model bietet mehr Flexibilität bei der Anpassung deines Fehlerkorrektur-Workflows. Weitere Informationen findest du im [Directed execution model](/guides/directed-execution-model)-Guide.
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit-ibm-runtime~=0.43.1
```
</details>
Fehlerkorrektur- und Fehlerunterdrückungstechniken werden verwendet, um die Ergebnisqualität beim Skalieren auf größere Workloads zu verbessern. Diese Seite bietet High-Level-Erklärungen der über Qiskit Runtime verfügbaren Fehlerunterdrückungs- und Fehlerkorrektur-Techniken.

Die folgende Zelle importiert das Estimator-Primitiv und erstellt ein Backend, das für die Initialisierung des Estimators in späteren Code-Zellen verwendet wird.

In [1]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy()

## Dynamical Decoupling

Quantenschaltungen werden auf IBM&reg;-Hardware als Sequenzen von Mikrowellenpulsen ausgeführt, die zu präzisen Zeitintervallen geplant und ausgeführt werden müssen.
Leider können unerwünschte Interaktionen zwischen Qubits zu kohärenten Fehlern auf inaktiven Qubits führen. Dynamical Decoupling funktioniert, indem Pulssequenzen auf inaktive Qubits eingefügt werden, um die Wirkung dieser Fehler annähernd zu kompensieren. Jede eingefügte Pulssequenz entspricht einer Identitätsoperation, aber die physische Anwesenheit der Pulse hat die Wirkung, Fehler zu unterdrücken.
Es gibt viele mögliche Auswahlmöglichkeiten für Pulssequenzen, und welche Sequenz für jeden speziellen Fall besser ist, bleibt ein [aktives Forschungsgebiet](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.20.064027).

Beachte, dass Dynamical Decoupling hauptsächlich für Schaltungen nützlich ist, die Lücken enthalten, in denen einige Qubits ohne Operationen inaktiv sind. Wenn die Operationen in der Schaltung sehr dicht gepackt sind, sodass alle Qubits die meiste Zeit beschäftigt sind, verbessert das Hinzufügen von Dynamical Decoupling-Pulsen möglicherweise nicht die Leistung. Tatsächlich könnte es die Leistung sogar aufgrund von Unvollkommenheiten in den Pulsen selbst verschlechtern.

Das untenstehende Diagramm zeigt Dynamical Decoupling mit einer XX-Pulssequenz. Die abstrakte Schaltung links wird auf einen Mikrowellenpuls-Plan oben rechts abgebildet. Die untere rechte Seite zeigt denselben Plan, jedoch mit einer Sequenz von zwei X-Pulsen, die während einer Leerlaufperiode des ersten Qubits eingefügt werden.

![Depiction of dynamical decoupling](../docs/images/guides/error-mitigation-explanation/dd.avif)

Dynamical Decoupling kann aktiviert werden, indem `enable` in den [dynamical decoupling options](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-dynamical-decoupling-options) auf `True` gesetzt wird. Die Option `sequence_type` kann verwendet werden, um aus mehreren verschiedenen Pulssequenzen auszuwählen. Der Standard-Sequenztyp ist `"XX"`.

Die folgende Code-Zelle zeigt, wie Dynamical Decoupling für den Estimator aktiviert und eine Dynamical Decoupling-Sequenz ausgewählt wird.

In [2]:
estimator = Estimator(mode=backend)
estimator.options.dynamical_decoupling.enable = True
estimator.options.dynamical_decoupling.sequence_type = "XpXm"

## Pauli Twirling

Twirling, auch bekannt als [randomized compiling](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.052325), ist eine weit verbreitete Technik zur Umwandlung beliebiger Rauschkanäle in Rauschkanäle mit spezifischerer Struktur.

Pauli Twirling ist eine spezielle Art von Twirling, die Pauli-Operationen verwendet. Es hat die Wirkung, jeden Quantenkanal in einen Pauli-Kanal zu transformieren. Allein durchgeführt kann es kohärentes Rauschen mildern, da kohärentes Rauschen dazu neigt, quadratisch mit der Anzahl der Operationen zu akkumulieren, während Pauli-Rauschen linear akkumuliert. Pauli Twirling wird oft mit anderen Fehlerkorrektur-Techniken kombiniert, die besser mit Pauli-Rauschen als mit beliebigem Rauschen funktionieren.

Pauli Twirling wird implementiert, indem ein ausgewählter Satz von Gates mit zufällig ausgewählten Einzel-Qubit-Pauli-Gates umgeben wird, sodass die ideale Wirkung des Gates gleich bleibt. Das Ergebnis ist, dass eine einzelne Schaltung durch ein zufälliges Ensemble von Schaltungen ersetzt wird, alle mit der gleichen idealen Wirkung. Beim Sampling der Schaltung werden Samples aus mehreren zufälligen Instanzen gezogen, anstatt nur aus einer einzigen.

![Depiction of Pauli twirling](../docs/images/guides/error-mitigation-explanation/pauli_twirling.avif)

Da die meisten Fehler in aktueller Quanten-Hardware von Zwei-Qubit-Gates stammen, wird diese Technik oft ausschließlich auf (native) Zwei-Qubit-Gates angewendet. Das folgende Diagramm zeigt einige Pauli Twirls für die CNOT- und ECR-Gates. Jede Schaltung innerhalb einer Zeile hat die gleiche ideale Wirkung.

![Depiction of gate twirls](../docs/images/guides/error-mitigation-explanation/gate_twirls.avif)

Pauli Twirling kann aktiviert werden, indem `enable_gates` in den [twirling options](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-twirling-options) auf `True` gesetzt wird. Weitere bemerkenswerte Optionen sind:

- `num_randomizations`: Die Anzahl der Schaltungsinstanzen, die aus dem Ensemble der getwirten Schaltungen gezogen werden sollen.
- `shots_per_randomization`: Die Anzahl der Shots, die aus jeder Schaltungsinstanz gezogen werden sollen.

Die folgende Code-Zelle zeigt, wie Pauli Twirling aktiviert und diese Optionen für den Estimator gesetzt werden. Keine dieser Optionen muss explizit gesetzt werden.

In [3]:
estimator = Estimator(mode=backend)
estimator.options.twirling.enable_gates = True
estimator.options.twirling.num_randomizations = 32
estimator.options.twirling.shots_per_randomization = 100

## Twirled Readout Error Extinction (TREX)

[Twirled readout error extinction (TREX)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.105.032620) mildert die Wirkung von Messfehlern für die Schätzung von Pauli-Observablen-Erwartungswerten.
Es basiert auf dem Konzept der getwirten Messungen, die durch zufällige Ersetzung von Mess-Gates durch eine Sequenz von (1) einem Pauli X-Gate, (2) einer Messung und (3) einem klassischen Bit-Flip erreicht werden. Genau wie beim Standard-Gate-Twirling ist diese Sequenz in Abwesenheit von Rauschen einer einfachen Messung äquivalent, wie im folgenden Diagramm dargestellt:

![Depiction of measurement twirling](../docs/images/guides/error-mitigation-explanation/measurement_twirling.avif)

Bei Vorhandensein von Auslesefehler hat Mess-Twirling die Wirkung, die Auslesefehler-Transfer-Matrix zu diagonalisieren, was es leicht macht, sie zu invertieren. Die Schätzung der Auslesefehler-Transfer-Matrix erfordert die Ausführung zusätzlicher Kalibrierungsschaltungen, was einen kleinen Overhead einführt.

TREX kann aktiviert werden, indem `measure_mitigation` in den [Qiskit Runtime resilience options](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2) für Estimator auf `True` gesetzt wird. Optionen für das Lernen von Messrauschen werden [hier](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-measure-noise-learning-options) beschrieben. Wie beim Gate-Twirling kannst du die Anzahl der Schaltungsrandomisierungen und die Anzahl der Shots pro Randomisierung festlegen.

Die folgende Code-Zelle zeigt, wie TREX aktiviert und diese Optionen für den Estimator gesetzt werden. Keine dieser Optionen muss explizit gesetzt werden.

In [4]:
estimator = Estimator(mode=backend)
estimator.options.resilience.measure_mitigation = True
estimator.options.resilience.measure_noise_learning.num_randomizations = 32
estimator.options.resilience.measure_noise_learning.shots_per_randomization = 100

<span id="zne"></span>
## Zero-Noise Extrapolation (ZNE)

Zero-Noise Extrapolation (ZNE) ist eine Technik zur Fehlerkorrektur bei der Schätzung von Erwartungswerten von Observablen. Obwohl sie oft die Ergebnisse verbessert, ist nicht garantiert, dass sie ein unvoreingenommenes Ergebnis liefert.

ZNE besteht aus zwei Stufen:

1. _Rauschverstärkung_: Die ursprüngliche Quantenschaltung wird mehrmals mit unterschiedlichen Rauschpegeln ausgeführt.
2. _Extrapolation_: Das ideale Ergebnis wird geschätzt, indem die verrauschten Erwartungswert-Ergebnisse zur Null-Rausch-Grenze extrapoliert werden.

Sowohl die Rauschverstärkungs- als auch die Extrapolationsstufe können auf viele verschiedene Arten implementiert werden. Qiskit Runtime implementiert Rauschverstärkung durch "digitales Gate-Folding", was bedeutet, dass Zwei-Qubit-Gates durch äquivalente Sequenzen des Gates und seiner Inversen ersetzt werden. Beispielsweise würde das Ersetzen eines unitären $U$ durch $U U^\dagger U$ einen Rauschverstärkungsfaktor von 3 ergeben. Für die Extrapolation kannst du aus einer von mehreren funktionalen Formen wählen, einschließlich einer linearen Anpassung oder einer exponentiellen Anpassung.
Das untenstehende Bild zeigt digitales Gate-Folding links und das Extrapolationsverfahren rechts.

![Depiction of ZNE](../docs/images/guides/error-mitigation-explanation/zne.avif)

ZNE kann aktiviert werden, indem `zne_mitigation` in den [Qiskit Runtime resilience options](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2) für Estimator auf `True` gesetzt wird.
Die Qiskit Runtime-Optionen für ZNE werden [hier](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-zne-options) beschrieben. Folgende Optionen sind bemerkenswert:

- `noise_factors`: Die Rauschfaktoren, die für die Rauschverstärkung verwendet werden sollen.
- `extrapolator`: Die funktionale Form, die für die Extrapolation verwendet werden soll.

Die folgende Code-Zelle zeigt, wie ZNE aktiviert und diese Optionen für den Estimator gesetzt werden. Keine dieser Optionen muss explizit gesetzt werden.

In [5]:
estimator = Estimator(mode=backend)
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = "exponential"

<span id="pea"></span>
## Probabilistic Error Amplification (PEA)

Eine der Hauptherausforderungen bei ZNE besteht darin, das Rauschen, das die Zielschaltung beeinflusst, genau zu verstärken. Gate-Folding bietet eine einfache Möglichkeit, diese Verstärkung durchzuführen, ist jedoch potenziell ungenau und kann zu falschen Ergebnissen führen. Siehe den Artikel ["Scalable error mitigation for noisy quantum circuits produces competitive expectation values"](https://arxiv.org/pdf/2108.09197), und insbesondere Seite 4 der ergänzenden Informationen für Details. Probabilistic Error Amplification bietet einen genaueren Ansatz zur Fehlerverstärkung durch Noise Learning.

PEA ist eine raffiniertere Technik, die vorbereitende Experimente durchführt, um das Rauschen zu rekonstruieren, und diese Informationen dann verwendet, um eine genaue Verstärkung durchzuführen. Sie beginnt mit dem Lernen des getwirten Rauschmodells jeder Schicht von Verschränkungs-Gates in der Schaltung, bevor sie ausgeführt werden (siehe [LayerNoiseLearningOptions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-layer-noise-learning-options) für relevante Lernoptionen). Nach der Lernphase werden die Schaltungen bei jedem Rauschfaktor ausgeführt, wobei jede Verschränkungs-Schicht der Schaltungen verstärkt wird, indem probabilistisch Einzel-Qubit-Rauschen proportional zum entsprechenden gelernten Rauschmodell injiziert wird. Siehe den Artikel ["Evidence for the utility of quantum computing before fault tolerance"](https://www.nature.com/articles/s41586-023-06096-3) für weitere Details.

PEA besteht aus drei Stufen:
1. _Lernen_: Das getwirte Rauschmodell jeder Schicht von Verschränkungs-Gates in der Schaltung wird gelernt.
1. _Rauschverstärkung_: Die ursprüngliche Quantenschaltung wird mehrmals mit unterschiedlichen Rauschfaktoren ausgeführt.
2. _Extrapolation_: Das ideale Ergebnis wird geschätzt, indem die verrauschten Erwartungswert-Ergebnisse zur Null-Rausch-Grenze extrapoliert werden.

Für Utility-Scale-Experimente ist PEA oft die beste Wahl.

Da PEA eine ZNE-Rauschverstärkungstechnik ist, musst du auch ZNE aktivieren, indem du `resilience.zne_mitigation = True` setzt. Andere [`resilience.zne`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-zne-options)-Optionen können zusätzlich verwendet werden, um Extrapolatoren, Verstärkungsstufen usw. festzulegen. PEA erfordert ein Rauschmodell, das bei Verwendung von Primitiven automatisch generiert wird.

Der folgende Codeausschnitt zeigt ein Beispiel, bei dem PEA verwendet wird, um das Ergebnis eines Estimator-Jobs zu korrigieren:

In [6]:
estimator = Estimator(mode=backend)
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.amplifier = "pea"

<span id="pec"></span>
## Probabilistic Error Cancellation (PEC)

Probabilistic Error Cancellation (PEC) ist eine Technik zur Fehlerkorrektur bei der Schätzung von Erwartungswerten von Observablen. Im Gegensatz zu ZNE liefert sie eine unvoreingenommene Schätzung des Erwartungswerts. Sie verursacht jedoch im Allgemeinen einen größeren Overhead.

Bei PEC wird die Wirkung einer idealen Zielschaltung als Linearkombination von verrauschten Schaltungen ausgedrückt, die in der Praxis tatsächlich implementierbar sind:

$$
\mathcal{O}_{\text{ideal}} = \sum_{i} \eta_i \mathcal{O}_{noisy, i}
$$

Die Ausgabe der idealen Schaltung kann dann reproduziert werden, indem verschiedene verrauschte Schaltungsinstanzen ausgeführt werden, die aus einem durch die Linearkombination definierten zufälligen Ensemble gezogen werden. Wenn die Koeffizienten $\eta_i$ eine Wahrscheinlichkeitsverteilung bilden, können sie direkt als Wahrscheinlichkeiten des Ensembles verwendet werden. In der Praxis sind einige der Koeffizienten negativ, sodass sie stattdessen eine Quasi-Wahrscheinlichkeitsverteilung bilden. Sie können immer noch verwendet werden, um ein zufälliges Ensemble zu definieren, aber es gibt einen Sampling-Overhead im Zusammenhang mit der Negativität der Quasi-Wahrscheinlichkeitsverteilung, der durch die Größe charakterisiert wird

$$
\gamma = \sum_{i} \lvert \eta_i \rvert \geq 1.
$$

Der Sampling-Overhead ist ein multiplikativer Faktor für die Anzahl der Shots, die erforderlich sind, um einen Erwartungswert auf eine gegebene Präzision zu schätzen, verglichen mit der Anzahl der Shots, die von der idealen Schaltung benötigt würden. Er skaliert quadratisch mit $\gamma$, was wiederum exponentiell mit der Tiefe der Schaltung skaliert.

PEC kann aktiviert werden, indem `pec_mitigation` in den [Qiskit Runtime resilience options](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2) für Estimator auf `True` gesetzt wird.
Die Qiskit Runtime-Optionen für PEC werden [hier](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-pec-options) beschrieben. Eine Begrenzung des Sampling-Overheads kann mit der Option `max_overhead` festgelegt werden. Beachte, dass die Begrenzung des Sampling-Overheads dazu führen kann, dass die Präzision des Ergebnisses die angeforderte Präzision überschreitet. Der Standardwert von `max_overhead` ist 100.

Die folgende Code-Zelle zeigt, wie PEC aktiviert und die Option `max_overhead` für den Estimator gesetzt wird.

In [7]:
estimator = Estimator(mode=backend)
estimator.options.resilience.pec_mitigation = True
estimator.options.resilience.pec.max_overhead = 100

## Nächste Schritte
- Sieh dir das [Tutorial](/tutorials/combine-error-mitigation-techniques) zur Kombination von Fehlerkorrektur-Optionen mit dem Estimator-Primitiv an.
- [Konfiguriere Fehlerkorrektur.](configure-error-mitigation)
- [Konfiguriere Fehlerunterdrückung.](configure-error-suppression)
- Erkunde andere [Optionen](runtime-options-overview) für die Qiskit Runtime-Primitive.
- Entscheide, in welchem [Ausführungsmodus](execution-modes) du deinen Job ausführen möchtest.